# Stage 3: Achieve higher derivatives

这个阶段的主要目标是扩展 DeZero，使其能够计算高阶导数。

In [2]:
import numpy as np
from dezero import Variable
from dezero import Function
import os
import subprocess
import math
from dezero.utils import plot_dot_graph

## Step 25: Visualization of computational graphs Ⅰ

使用Graphviz库，我们可以将计算图可视化为图形。这样可以更直观地理解计算图的结构。

## Step 26: Visualization of computational graphs Ⅱ

In [3]:
# 将Variable实例赋给函数，返回DOT语言编写的表示实例信息的字符串
def _dot_var(v, verbose=False):
    dot_var = '{} [label="{}", color=orange, style=filled]\n'
    name = '' if v.name is None else v.name
    # verbose为True时，添加变量的形状和数据类型
    if verbose and v.data is not None:
        if v.name is not None:
            name += ': '
        name += str(v.shape) + ' ' + str(v.dtype)
    return dot_var.format(id(v), name)  # 使用python内置函数id()获取对象的内存地址

In [5]:
x = Variable(np.random.randn(2, 3))
x.name = 'x'
print(_dot_var(x))
print(_dot_var(x, verbose=True))

1402407802760 [label="x", color=orange, style=filled]

1402407802760 [label="x: (2, 3) float64", color=orange, style=filled]



In [4]:
# 将Function实例赋给函数，返回DOT语言编写的表示实例信息的字符串
def _dot_func(f):
    dot_func = '{} [label="{}", color=lightblue, style=filled, shape=box]\n'
    txt = dot_func.format(id(f), f.__class__.__name__)
    # 用DOT记述函数与输入变量之间的连接，以及函数与输出变量之间的连接
    dot_edge = '{} -> {}\n' 
    for x in f.inputs:
        txt += dot_edge.format(id(x), id(f))
    for y in f.outputs:
        txt += dot_edge.format(id(f), id(y()))  # y是weakref
    return txt

In [6]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
y = x0 + x1
txt = _dot_func(y.creator)
print(txt)

1404453256520 [label="Add", color=lightblue, style=filled, shape=box]
1402407784328 -> 1404453256520
1402407783624 -> 1404453256520
1404453256520 -> 1404287120264



In [7]:
# 参考backward函数的实现，实现获取计算图的函数
def get_dot_graph(output, verbose=True):
    txt = ''
    funcs = []
    seen_set = set()

    def add_func(f):
        if f not in seen_set:  # 不需要按辈分顺序添加函数
            funcs.append(f)
            seen_set.add(f)

    add_func(output.creator)
    txt += _dot_var(output, verbose)  # 输出变量

    while funcs:
        func = funcs.pop()
        txt += _dot_func(func)  # 函数
        for x in func.inputs:
            txt += _dot_var(x, verbose)  # 输入变量

            if x.creator is not None:
                add_func(x.creator)  # 添加函数

    return 'digraph g {\n' + txt + '}'

In [13]:
def plot_dot_graph(output, verbose=True, to_file='graph.png'):
    dot_graph = get_dot_graph(output, verbose)

    # 保存为dot文件
    tmp_dir = os.path.join(os.path.expanduser('~'), '.dezero')
    if not os.path.exists(tmp_dir):  # 创建目录
        os.mkdir(tmp_dir)
    dot_file = os.path.join(tmp_dir, 'tmp.dot')
    with open(dot_file, 'w') as f:
        f.write(dot_graph)

    # 调用Graphviz的dot命令
    ext = os.path.splitext(to_file)[1][1:]  # 获取文件扩展名
    cmd = 'dot {} -T {} -o {}'.format(dot_file, ext, to_file)
    subprocess.run(cmd, shell=True)

    # 在juptyer notebook中显示图像
    try:
        from PIL import Image
        img = Image.open(to_file)
        img.show()
    except ImportError:
        pass

In [11]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
y = x0 + x1
x0.name = 'x0'
x1.name = 'x1'
y.name = 'y'
plot_dot_graph(y, verbose=False, to_file='add.png')

In [15]:
def goldstein(x, y):
    z = (1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2)) * \
        (30 + (2*x - 3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return z


x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = goldstein(x, y)
z.backward()

x.name = 'x'
y.name = 'y'
z.name = 'z'
plot_dot_graph(z, verbose=False, to_file='goldstein.png')

## Step 27: Derivative of Taylor's expansion

In [18]:
class Sin(Function):
    # 直接使用np.sin()计算正弦函数的值，np.cos()计算导数
    def forward(self, x):
        y = np.sin(x)
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = gy * np.cos(x)
        return gx
    
def sin(x):
    return Sin()(x)

In [19]:
x = Variable(np.array(np.pi/4))
y = sin(x)
y.backward()

print(y.data)
print(x.grad)

0.7071067811865476
0.7071067811865476


使用泰勒展开逼近函数：

$$
f(x) = f(a) + f'(a)(x-a) + \frac{f''(a)}{2!}(x-a)^2 + \frac{f'''(a)}{3!}(x-a)^3 + \cdots
$$

a = 0时，泰勒展开式称为麦克劳林级数。
$$
f(x) = f(0) + f'(0)x + \frac{f''(0)}{2!}x^2 + \frac{f'''(0)}{3!}x^3 + \cdots
$$

对于sinx，sin(0) = 0，sin'(0) = cos(0) = 1，sin''(0) = -sin(0) = 0，sin'''(0) = -cos(0) = -1，以此类推可得：
$$
\sin x = x - \frac{x^3}{3!} + \frac{x^5}{5!} - \frac{x^7}{7!} + \cdots
$$

In [21]:
def my_sin(x, threshold=1e-4):
    y = 0
    for i in range(100000):
        c = (-1) ** i / math.factorial(2*i+1)
        t = c * x ** (2*i+1)
        y = y + t
        if abs(t.data) < threshold:
            break
    return y

In [22]:
x = Variable(np.array(np.pi/4))
y = my_sin(x)
y.backward()

print(y.data)
print(x.grad)

0.7071064695751781
0.7071032148228457


In [24]:
plot_dot_graph(y, verbose=False, to_file='my_sin.png')

## Step 28: Function optimization

DeZero 现在能够自动计算导数了。导数有多种用途，其中最重要的一种用途就是函数优化。
 
优化指的是对于给定的函数，找到使其取得最大值或最小值的函数参数或输入值。

本步骤处理Rosenbrock函数，又叫香蕉函数，目标是找到使得Rosenbrock函数取得最小值的x0和x1。

In [26]:
def rosenbrock(x0, x1):
    y = 100 * (x1 - x0 ** 2) ** 2 + (1 - x0) ** 2
    return y

In [ ]:
# 使用梯度下降法求解Rosenbrock函数的最小值
x0 = Variable(np.array(0.0))
x1 = Variable(np.array(2.0))
lr = 0.001
iters = 1000

for i in range(iters):
    print(x0, x1)
    y = rosenbrock(x0, x1)
    x0.cleargrad()
    x1.cleargrad()
    y.backward()

    x0.data -= lr * x0.grad
    x1.data -= lr * x1.grad
# 很多次迭代后，x0和x1的值接近于1，1

## Step 29: Optimization using Newton method

上一个步骤使用了很多次迭代梯度下降法才找到最终的解，收敛速度较慢。（梯度下降本身也不擅长处理rosenbrock函数，因为rosenbrock函数的梯度在接近最小值时会变得很小，导致梯度下降法收敛速度变慢。）

本步骤使用牛顿法，能以更少的步数获取最优解。
<center>
<table>
  <tr>
    <td><img src="./res/difference.png" width="500"/></td>
    <td><img src="./res/牛顿法.png" width="300"/></td>
  </tr>
</table>
</center>

二次近似：（截断泰勒展开式到二次项）
$$
f(x) \approx f(x_0) + f'(x_0)(x-x_0) + \frac{1}{2}f''(x_0)(x-x_0)^2
$$
二次近似函数的最小值位于：$x = x_0 - \frac{f'(x_0)}{f''(x_0)}$

梯度下降法，牛顿法的迭代公式分别为：
$$
x_{new} = x_{old} - \eta f'(x_{old}) \\
x_{new} = x_{old} - \frac{f'(x_{old})}{f''(x_{old})}
$$

可以将牛顿法看作时梯度下降法的改进版，通过二阶导数信息来调整学习率。


In [3]:
def f(x):
    y = x ** 4 - 2 * x ** 2
    return y

def gx2(x):  # 手动计算二阶导数
    return 12 * x ** 2 - 4

x = Variable(np.array(2.0))
iters = 10

for i in range(iters):
    print(i, x)
    y = f(x)
    x.cleargrad()
    y.backward()
    x.data -= x.grad / gx2(x.data)

0 variable(2.0)
1 variable(1.4545454545454546)
2 variable(1.1510467893775467)
3 variable(1.0253259289766978)
4 variable(1.0009084519430513)
5 variable(1.0000012353089454)
6 variable(1.000000000002289)
7 variable(1.0)
8 variable(1.0)
9 variable(1.0)


## Step 30: Achieving Higher Derivatives (Preparation)

复习Variable类和Function类的实现，为实现高阶导数做准备。
- 计算的连接是在 Function 类的__call__方法中创建的
- 正向传播和反向传播的具体计算是在继承了 Function 的类的forward法和 backward方法中进行的

## Step 31: Achieving Higher Derivatives (Theory)

连接是在进行正向传播的计算时创建的，在反向传播时不会被创建。因为该计算针对的是ndarray实例。

求导计算换句话说就是通过反向传播进行计算。如果能对反向传播所进行的计算创建连接，就能自动计算高阶导数。

为此，我们需要**将导数(梯度)保存为 Variable 实例**！

<center>
<table>
  <tr>
    <td><img src="./res/反向传播创建连接.png" width="300"/></td>
    <td><img src="./res/反向传播创建连接2.png" width="300"/></td>
  </tr>
</table>
</center>

## Step 32: Achieving Higher Derivatives (Implementation)